In [144]:
#| default_exp nb

In [145]:
#| export
from __future__ import annotations

# Notebook objects

> Notebook cells and outputs and helpers.



# Prologue

In [146]:
#| export

import collections
import inspect
import operator as op
from datetime import datetime
from functools import cache
from functools import partial
from typing import Callable
from typing import Iterable
from typing import Literal
from typing import Mapping
from typing import Protocol
from typing import runtime_checkable
from typing import Sequence
from typing import TypeAlias

import fastcore.all as FC
from fastcore.all import L
from fastcore.all import nested_idx
from nbdev.process import extract_directives
from nbdev.showdoc import add_docs
from olio.basic import empty
from olio.basic import gets
from olio.basic import pops_values_
from olio.callback import Callback
from olio.callback import CollBack
from olio.callback import FuncCB
from olio.common import AD
from olio.common import update_
from olio.common import vals_at
from rich.pretty import pretty_repr

In [147]:
#| export

from bridget.helpers import cached_property
from bridget.helpers import compose_first
from bridget.helpers import emptyd

In [148]:
import json
from pathlib import Path
from types import MethodType
from typing import Any
from typing import cast

import nbformat
import toolz as TZ
import traitlets as T
from fastcore.test import *
from IPython.display import display
from nbdev.showdoc import *
from olio.callback import process_
from olio.common import shorten
from olio.common import shortens
from olio.common import val_at
from rich.console import Console

In [149]:
import bridget
from bridget.display_helpers import RenderJSON

----


In [150]:
cprint = (console := Console(width=120)).print

----

In [151]:
#| export

def ts(): return f"{datetime.now():%H:%M:%S.%f}"[:-3]

In [152]:
ts()

'15:04:13.167'

# Notebook data


```javascript
{
    "type": "state",  // "state" | ...
    "cells": [
        // Markdown cell
        {
            "cell_type": 1,  // 1: markdown, 2: code
            "source": "string",  // Cell content
            "metadata"?: {  // Optional
                "tags": ["string"],
                "jupyter": { /* jupyter specific */ },
                "brd": "string"
            }
        },
        // Code cell with outputs
        {
            "cell_type": 2,
            "source": "string",
            "outputs"?: [  // Optional, only for code cells
                {
                    "output_type": 1 | 2 | 3 | 4 // "stream" | "display_data" | "execute_result" | "error",
                    "metadata"?: {...},  // Optional
                    // Type-specific fields
                    "name"?: "stdout" | "stderr",  // For stream
                    "text"?: "string",             // For stream
                    "data"?: {  // Optional
                        "mime/type": "string"  // e.g., "text/plain": "content"
                    },
                    "ename"?: "string",            // For error
                    "evalue"?: "string",           // For error
                    "traceback"?: ["string"],      // For error
                    "execution_count"?: number     // For execute_result
                }
            ]
        }
    ]
}
```

In [153]:
state = json.loads(Path('../packages/nbinspect-vscode/test/outputs.json').read_text('utf-8'))

# cprint(state, width=120, overflow='ellipsis')
RenderJSON(state, init_level=4, max_height=400).display()

In [154]:
def f(c):
	c = c.copy()
	c['id'] = str(c['idx'])
	del c['idx']
	# if c['cell_type'] == 'code':
	# 	if 'outputs' not in c: 
	# 		c['outputs'] = []
	# 	else:
	# 		for o in c['outputs']:
	# 			if o['output_type'] == 'error':
	# 				if 'metadata' in o: del o['metadata']
	# 	if 'execution_count' not in c: c['execution_count'] = None
	return c

cells = map(f, state['cells'])
nb = {
	'cells': list(cells)[:],
	'metadata': {
		'kernel_info': state['nbData']['metadata']['metadata']['kernelspec'],
		'language_info': state['nbData']['metadata']['metadata']['language_info'],
		},
	'nbformat': state['nbData']['metadata']['nbformat'],
	'nbformat_minor': 5 # state['nbData']['metadata']['nbformat_minor']
}
RenderJSON(nb, init_level=2, max_height=400).display()

In [155]:
nb = nbformat.reads(json.dumps(nb), as_version=nbformat.NO_CONVERT, capture_validation_error=(derr := {}))
test_eq(derr, {})

# NBCell

Note: though using `AD` for convenience, `NBCell` should be considered immutable.


In [156]:
#| export

CellTypesT: TypeAlias = Literal['code', 'markdown']

copycell = op.methodcaller('copy')

def _relevant_kw(o, kw:Mapping):
    ks = tuple(FC.flatten(map(inspect.get_annotations, type(o).mro())))
    return FC.filter_keys(kw, FC.in_(ks))  # type: ignore


class NBCell(AD, metaclass=FC.NewChkMeta):
    idx: str
    cell_type: str
    source: str
    metadata: dict
    def __new__(cls, cell: Mapping):
        if cls is NBCell: cls = _CTYP[cell.get('cell_type', 'raw')]
        return AD.__new__(cls, cell)
    def __init__(self, cell: Mapping):
        super().__init__(_relevant_kw(self, cell))
        self.source = ''.join(getattr(self, 'source', ()))
    def _repr_markdown_(self): 
        return f'> {self.get('cell_type', 'raw')}\n```json\n{pretty_repr(self, indent_size=2, max_width=120)}\n```'
#     def _repr_markdown_(self): 
#         return f"""
# <details><summary>{self.cell_type}</summary>

# ```json\n{pretty_repr(self, indent_size=2, max_width=120)}\n```

# </details>"""
    
    def copy(self) -> NBCell: return NBCell(dict(self))
    
    @cached_property
    def directives_(self) -> dict[str, list[str]]: return extract_directives(self, False)
    def has_directive(self, directive: str, *args): return has_directive(self, directive, *args)
    @cached_property
    def hidden(self): return has_directive(self, 'hide') or has_directive(self, 'include', 'false')


class NBCellRaw(NBCell): attachments: dict
class NBCellMarkdown(NBCellRaw):...
class NBCellCode(NBCell): 
    outputs: tuple[NBOutput, ...]
    execution_count: int | None
    def __init__(self, cell: Mapping):
        super().__init__(cell)
        self.outputs = tuple(NBOutput(o) for o in cell.get('outputs', ()))


class NBOutput(AD, metaclass=FC.NewChkMeta): 
    output_type: str
    def __new__(cls, out: Mapping):
        if cls is NBOutput: cls = _OUTTYP[out.get('output_type', 'stream')]
        return AD.__new__(cls, out)
    def __init__(self, out: Mapping):
        ks = tuple(FC.flatten(map(inspect.get_annotations, type(self).mro())))
        out = FC.filter_keys(out, FC.in_(ks))  # type: ignore
        super().__init__(out)

class NBOutputStream(NBOutput):
    name: Literal['stdout', 'stderr']
    text: str

class NBOutputDisplayData(NBOutput):
    data: dict
    metadata: dict

class NBOutputExecuteResult(NBOutput):
    execution_count: int
    data: dict
    metadata: dict

class NBOutputError(NBOutput):
    ename: str
    evalue: str
    traceback: list[str]


_CTYP = {'raw':NBCellRaw, 'markdown':NBCellMarkdown, 'code':NBCellCode}
# _CTYPInv = {v:k for k,v in _CTYP.items()}
_OUTTYP = {'stream':NBOutputStream, 'display_data':NBOutputDisplayData, 'execute_result':NBOutputExecuteResult, 'error':NBOutputError}
# _OUTTYPInv = {v:k for k,v in _OUTTYP.items()}

def has_directive(c: NBCell, directive: str, *args):
    return directive in c.directives_ and c.directives_[directive] == list(args)

In [157]:
test_eq(NBCell({}), NBCellRaw({}))
test_eq(NBCell(AD(cell_type='markdown')), {'cell_type': 'markdown', 'source': ''})
test_eq(
    NBCell(AD(cell_type='code', source='print("hello")')), 
    {'cell_type': 'code', 'source': 'print("hello")', 'outputs': ()})
test_eq(
    NBCell(
        AD(cell_type='code', source='display("hello")', 
            outputs=[AD(output_type='display_data', data={'text/plain': 'hello'})])),
    {'cell_type': 'code', 'source': 'display("hello")', 'outputs': (AD(output_type='display_data', data={'text/plain': 'hello'}),)})
test_eq(
    NBCell({
        'cell_type': 'code',
        'source': "display(HTML('cell 4'))\n",
        'outputs': ({
            'output_type': 'display_data',
            'data': {'text/html': 'cell 4', 'text/plain': '<IPython.core.display.HTML object>'},
            },)}),
    {'cell_type': 'code', 'source': "display(HTML('cell 4'))\n", 'outputs': (AD(output_type='display_data', data={'text/html': 'cell 4', 'text/plain': '<IPython.core.display.HTML object>'}),)})

In [158]:
c1 = NBCell(dict(cell_type='code', source='display("hello")'))
c2 = NBCell(c1)
test_is(c1, c2)

c3 = c2.copy()
test_is(c1 is c3, False)
test_eq(c1.source, c3.source)
test_eq(c1.outputs, c3.outputs)

In [159]:
NBCell(state['cells'][1])

> code
```json
{
  'idx': 1,
  'cell_type': 'code',
  'source': '# cell 1\nfrom itertools import count\n\nimport ipywidgets as W\nimport matplotlib.pyplot as plt\nfrom bridget.display_helpers import displaydh\nfrom IPython.display import HTML, Image, Javascript, JSON, DisplayHandle, clear_output\ncounter = count()',
  'metadata': {'brd': 'ca92cce1-b16c-487b-9869-899330cec0ac', 'cell_id': 'W1sZmlsZQ=='},
  'outputs': (),
  'execution_count': 1
}
```

In [160]:
cell = NBCell(state['cells'][4])
test_is(type(cell), NBCellCode)
cell

> code
```json
{
  'idx': 4,
  'cell_type': 'code',
  'source': "# cell 4\ndisplaydh(HTML('cell 4'), metadata={'bridge': {'cell': 4}});",
  'metadata': {'brd': '425ba617-db53-4918-958e-1a355849a808', 'cell_id': 'W4sZmlsZQ=='},
  'outputs': (
    {
      'output_type': 'display_data',
      'data': {'text/html': 'cell 4', 'text/plain': '<IPython.core.display.HTML object>'},
      'metadata': {'transient': {'display_id': '551c39d61ff87c17a1e7b0e0bfae863d'}, 'bridge': {'cell': 4}}
    },
  ),
  'execution_count': 4
}
```

In [161]:
nested_idx(cell, 'outputs', 0, 'metadata', 'transient', 'display_id')

'551c39d61ff87c17a1e7b0e0bfae863d'

In [162]:
test_eq(nested_idx(NBCell(state['cells'][0]), 'outputs', 0, 'metadata', 'transient', 'display_id'), None)

In [163]:
nested_idx(getattr(NBCell(state['cells'][3]), 'outputs'), 0, 'metadata', 'transient', 'display_id')

'9fd35d8abc854b3d601517c52d79350c'

## did

In [164]:
#| export

def did(o:NBOutputDisplayData) -> str|None: 
    return nested_idx(o, 'metadata', 'transient', 'display_id')  # type: ignore

FC.patch(did, as_prop=True)

In [165]:
# type: ignore

display(cell.outputs[0])

test_eq(did(cell.outputs[0]), cell.outputs[0]['metadata']['transient']['display_id'])
test_eq(cell.outputs[0].did, cell.outputs[0]['metadata']['transient']['display_id'])

```json
{ 'data': { 'text/html': 'cell 4',
            'text/plain': '<IPython.core.display.HTML object>'},
  'metadata': { 'bridge': {'cell': 4},
                'transient': { 'display_id': '551c39d61ff87c17a1e7b0e0bfae863d'}},
  'output_type': 'display_data'}
```

## dids

In [166]:
#| export

def dids(o:NBCell)->L[str|None]: return L(o.outputs).map(did)  # type: ignore

FC.patch(dids, as_prop=True)

In [167]:
# type: ignore

cell = NBCell(state['cells'][5])
display(dids(cell))
cell.dids

(#1) ['a2505dd6c88c3bb2f50fee94e8168fa4']

(#1) ['a2505dd6c88c3bb2f50fee94e8168fa4']

In [168]:
display(cell := NBCell(state['cells'][7]))
cell.dids

> code
```json
{
  'idx': 7,
  'cell_type': 'code',
  'source': "# cell 7\ndisplaydh(HTML('cell 7.1'), metadata={'bridge': {'cell': 7.1}})\ndisplaydh(HTML('cell 7.2'), metadata={'bridge': {'cell': 7.2}});",
  'metadata': {'brd': 'e43f0a76-d4e3-499f-900a-390b4b877044', 'cell_id': 'X10sZmlsZQ=='},
  'outputs': (
    {
      'output_type': 'display_data',
      'data': {'text/html': 'cell 7.1', 'text/plain': '<IPython.core.display.HTML object>'},
      'metadata': {'transient': {'display_id': 'eb8906f41a8c5b15e9d09ec36cadf9bd'}, 'bridge': {'cell': 7.1}}
    },
    {
      'output_type': 'display_data',
      'data': {'text/html': 'cell 7.2', 'text/plain': '<IPython.core.display.HTML object>'},
      'metadata': {'transient': {'display_id': '0b3856420dda423aa16aa308054a5353'}, 'bridge': {'cell': 7.2}}
    }
  ),
  'execution_count': 7
}
```

(#2) ['eb8906f41a8c5b15e9d09ec36cadf9bd','0b3856420dda423aa16aa308054a5353']

In [169]:
cells = L(state['cells'])

In [170]:
test_eq(cells[2]['outputs'][0]['name'], 'stdout')  # type: ignore

In [171]:
test_eq(len(codes := cells.argwhere(lambda c: c['cell_type'] == 'code')), 38)
display(codes)
cells[codes[2]]

(#38) [1,2,3,4,5,6,7,8,9,10...]

{'idx': 3,
 'cell_type': 'code',
 'source': "# cell 3\ndisplaydh('cell 3', metadata={'bridge': {'cell': 3}});",
 'metadata': {'brd': 'asdf-qwer', 'cell_id': 'W3sZmlsZQ=='},
 'outputs': [{'output_type': 'display_data',
   'data': {'text/plain': "'cell 3'"},
   'metadata': {'transient': {'display_id': '9fd35d8abc854b3d601517c52d79350c'},
    'bridge': {'cell': 3}}}],
 'execution_count': 3}

## by_type

In [172]:
#| export

def by_type(cells: Sequence[NBCell]|L, cell_type: CellTypesT):
    return L(cells).argwhere(lambda c: c['cell_type'] == cell_type)

In [173]:
by_type(cells, 'code')

(#38) [1,2,3,4,5,6,7,8,9,10...]

## idx2cell

In [174]:
#| export

def idx2cell(cells: Sequence[NBCell]|L, cell_type: CellTypesT|None=None) -> dict[int, L]:
    cells = L(cells)
    idxs = by_type(cells, cell_type) if cell_type else L.range(len(cells))
    return dict(idxs.zipwith(cells[idxs]))

In [175]:
idx2cell(cells, 'code').keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39])

In [176]:
outputs_idx = cells.argwhere(lambda c: 'outputs' in c)
display(outputs_idx)

print(outputs_idx.zipwith(cells[outputs_idx])[0])

dict(outputs_idx.zipwith(cells[outputs_idx]))[3]

(#38) [1,2,3,4,5,6,7,8,9,10...]

(1, {'idx': 1, 'cell_type': 'code', 'source': '# cell 1\nfrom itertools import count\n\nimport ipywidgets as W\nimport matplotlib.pyplot as plt\nfrom bridget.display_helpers import displaydh\nfrom IPython.display import HTML, Image, Javascript, JSON, DisplayHandle, clear_output\ncounter = count()', 'metadata': {'brd': 'ca92cce1-b16c-487b-9869-899330cec0ac', 'cell_id': 'W1sZmlsZQ=='}, 'outputs': [], 'execution_count': 1})


{'idx': 3,
 'cell_type': 'code',
 'source': "# cell 3\ndisplaydh('cell 3', metadata={'bridge': {'cell': 3}});",
 'metadata': {'brd': 'asdf-qwer', 'cell_id': 'W3sZmlsZQ=='},
 'outputs': [{'output_type': 'display_data',
   'data': {'text/plain': "'cell 3'"},
   'metadata': {'transient': {'display_id': '9fd35d8abc854b3d601517c52d79350c'},
    'bridge': {'cell': 3}}}],
 'execution_count': 3}

## withOutputs

In [177]:
#| export

def withOutputs(cells: Sequence[NBCell]|L):
    'Return indices of cells with outputs'
    return L(cells).argwhere(lambda c: 'outputs' in c)

In [178]:
withOutputs(cells[:5])

(#4) [1,2,3,4]

## idx2outputs

In [179]:
#| export

def idx2outputs(cells: Sequence[NBCell]|L) -> dict[int, L]:
    'Return dict of indices to cells with outputs'
    cells = L(cells)
    outputs_idx = withOutputs(cells)
    return dict(outputs_idx.zipwith(cells[outputs_idx]))

In [180]:
idx2outputs(cells)[2]

{'idx': 2,
 'cell_type': 'code',
 'source': '# cell 2\nimport time\n# time.sleep(2)\nprint(1)',
 'metadata': {'brd': 'fea8ed66-7099-4d14-ae45-fb3d468bc7c0',
  'cell_id': 'W2sZmlsZQ=='},
 'outputs': [{'output_type': 'stream', 'name': 'stdout', 'text': '1\n'}],
 'execution_count': 2}

In [181]:
idx2outputs(cells)[7]

{'idx': 7,
 'cell_type': 'code',
 'source': "# cell 7\ndisplaydh(HTML('cell 7.1'), metadata={'bridge': {'cell': 7.1}})\ndisplaydh(HTML('cell 7.2'), metadata={'bridge': {'cell': 7.2}});",
 'metadata': {'brd': 'e43f0a76-d4e3-499f-900a-390b4b877044',
  'cell_id': 'X10sZmlsZQ=='},
 'outputs': [{'output_type': 'display_data',
   'data': {'text/html': 'cell 7.1',
    'text/plain': '<IPython.core.display.HTML object>'},
   'metadata': {'transient': {'display_id': 'eb8906f41a8c5b15e9d09ec36cadf9bd'},
    'bridge': {'cell': 7.1}}},
  {'output_type': 'display_data',
   'data': {'text/html': 'cell 7.2',
    'text/plain': '<IPython.core.display.HTML object>'},
   'metadata': {'transient': {'display_id': '0b3856420dda423aa16aa308054a5353'},
    'bridge': {'cell': 7.2}}}],
 'execution_count': 7}

In [182]:
idx2outputs(cells)[12]

{'idx': 12,
 'cell_type': 'code',
 'source': "# cell 12\nW.IntSlider(12, description='cell')",
 'metadata': {'brd': '8b29b413-e054-4a61-8ab9-db8031064cde',
  'cell_id': 'X15sZmlsZQ=='},
 'outputs': [{'output_type': 'execute_result',
   'data': {'application/vnd.jupyter.widget-view+json': {'version_major': 2,
     'version_minor': 0,
     'model_id': 'e167426b9a62409f9e67edc703a25e53'},
    'text/plain': "IntSlider(value=12, description='cell')"},
   'execution_count': 12,
   'metadata': {}}],
 'execution_count': 12}

## directives
> `NBCell`s register **nbdev [directives](https://nbdev.fast.ai/explanations/directives.html)**/**quarto [Cell options](https://quarto.org/docs/reference/cells/cells-jupyter.html)**/**python [cell magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cell-magics)** in `directives_`.

In [183]:
s = '''
%%python
#| hide

print(1)
'''
cell1 = NBCell(AD(cell_type='code', source=s))
test_is(cell1.has_directive('hide'), True)
cell1.directives_

{'python': [], 'hide': []}

In [184]:
s = '''
#| label: fig-polar
#| echo: false

# comment
print(1)
'''
cell2 = NBCell(dict(cell_type='code', source=s))
cell2.directives_

{'label:': ['fig-polar'], 'echo:': ['false']}

In [185]:
s = '''

    #| code-fold

print(1)
'''
cell3 = NBCell({'cell_type':'code', 'source':s})
cell3.directives_

{'code-fold': []}

In [186]:
#| export

def by_directive(cells: Sequence[NBCell]|L, directive: str, *args):
    return L(cells).argwhere(lambda c: has_directive(c, directive, *args))

In [187]:
test_eq(by_directive([cell1, cell2, cell3], 'hide'), [0])

# NB

In [188]:
#| export

@runtime_checkable
class NBProvider(Protocol):
    @property
    def nb(self) -> NB: ...

@runtime_checkable
class NBProcessor(Protocol):
    def __call__(self, nb:NB, *args, **kwargs) -> NB: ...


class NB(AD):
    "Bridget representation of notebook state"
    cells: L[NBCell]  # type: ignore
    nbData: dict
    type: str
    timestamp: str
    origin: str
    def __init__(self, cells: Sequence[NBCell]|L=(), **kwargs):
        self.cells = L(NBCell(_) for _ in cells)
        d = update_({'nbData':{}, 'type':'state', 'timestamp':'', 'origin':''}, **_relevant_kw(self, kwargs))
        super().__init__(**d)
    
    @classmethod
    def fromStateMessage(cls, message):
        message = message.copy()
        cells = pops_values_(message, 'cells')[0]
        return cls(cells, **message)
    
    def by_type(self, cell_type: CellTypesT): return by_type(self.cells, cell_type)
    @cached_property
    def codes(self): return by_type(self.cells, 'code')
    @cached_property
    def mds(self): return by_type(self.cells, 'markdown')

    def idx2cell(self, cell_type: CellTypesT|None=None): return idx2cell(self.cells, cell_type)
    @cached_property
    def idx2code(self): return idx2cell(self.cells, 'code')
    @cached_property
    def idx2md(self): return idx2cell(self.cells, 'markdown')

    @cached_property
    def withOutputs(self): return withOutputs(self.cells)
    @cached_property
    def idx2outputs(self) -> dict[int, L]: return idx2outputs(self.cells)

    def by_directive(self, directive: str, *args): return by_directive(self.cells, directive)
    @cached_property
    def hiddens(self): return self.by_directive('hide') + self.by_directive('include', 'false')

    @cache
    def select(self, k='source'): return self.cells.attrgot(k, None)

    def _process(self, items, cbs):
        collections.deque(CollBack(items, context=self, cbs=cbs), maxlen=0)
        return FC.first(cbs.attrgot('nb')[::-1]) or self
    
    def process(self, /,
            cbs: Callback|Sequence[Callback]=(), 
            slc:slice|None=None, 
            pred:Callable[[NBCell], bool]|None=None, 
            **kwargs  # FuncCB kwargs
        ) -> NB:
        "Process a subset `slc` of cells filtered by `pred` with `cbs` and `FuncCB` callbacks."
        _cbs = FC.L(FC.tuplify(cbs) + ((FuncCB(**kwargs),) if kwargs else ()))
        items = self.cells[slc or slice(None)].filter(pred) if pred or slc else self.cells  # type: ignore
        if _cbs: return self._process(items, _cbs)
        else: return NB(items, nbData=self.nbData, type=self.type, timestamp=self.timestamp)
    
    def pipe(self, funcs:Iterable[NBProcessor], *args, **kwargs) -> NB: 
        return compose_first(*funcs)(self, *args, **kwargs)  # type: ignore
    
    @classmethod
    def from_NB(cls, nb: NB, cells: Sequence[NBCell]|L|None=None, **kwargs): 
        return NB(cells if cells is not None else nb.cells, 
			**update_({'nbData':nb.nbData, 'type':nb.type, 'timestamp':nb.timestamp}, **kwargs))


for k in ('source', 'metadata', 'outputs'): setattr(NB, k, property(NB.select.__wrapped__,))

In [189]:
test_eq(NB(), {'cells': [], 'nbData': {}, 'type': 'state', 'timestamp': '', 'origin': ''})
test_eq(NB(nbData={'a':1}), {'cells': [], 'nbData':{'a':1}, 'type': 'state', 'timestamp': '', 'origin': ''})

In [190]:
nb = NB.fromStateMessage(state)
cells = nb.cells

test_eq(len(nb.cells), nb.nbData['cellCount'])
test_eq(nb.nbData['metadata']['nbformat'], 4)
nb.cells[2]

> code
```json
{
  'idx': 2,
  'cell_type': 'code',
  'source': '# cell 2\nimport time\n# time.sleep(2)\nprint(1)',
  'metadata': {'brd': 'fea8ed66-7099-4d14-ae45-fb3d468bc7c0', 'cell_id': 'W2sZmlsZQ=='},
  'outputs': ({'output_type': 'stream', 'name': 'stdout', 'text': '1\n'},),
  'execution_count': 2
}
```

In [191]:
#|export

add_docs(NB,
    by_type="Return indices of cells of type `cell_type`",
    codes="Cell indices of type `code`",
    mds="Cell indices of type `markdown`",
    idx2cell="Return dict of indices to cells of type `cell_type`",
    idx2code="Return dict of indices to cells of type `code`",
    idx2md="Return dict of indices to cells of type `markdown`",
    withOutputs="Return indices of cells with outputs",
    idx2outputs="Return dict of indices to cells with outputs",
    by_directive="Return indices of cells with directive `directive`",
    hiddens="Return indices of cells with `hide` or `include:false` directives",
    pipe="Pipe a sequence of `NBProcessor`s",
    from_NB="Create a new `NB` from an existing `NB`",
    select="Return `k` attribute of cells",
    source="Return `source` attribute of all cells",
    metadata="Return `metadata` attribute of all cells",
    outputs="Return `outputs` attribute of all cells",
)

In [192]:
show_doc(NB.by_type)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L210){target="_blank" style="float:right; font-size:smaller"}

### NB.by_type

>      NB.by_type (cell_type:Literal['code','markdown'])

*Return indices of cells of type `cell_type`*

In [193]:
display(nb.cells[2])
test_eq(cells[2].outputs[0].name, 'stdout')  # type: ignore

> code
```json
{
  'idx': 2,
  'cell_type': 'code',
  'source': '# cell 2\nimport time\n# time.sleep(2)\nprint(1)',
  'metadata': {'brd': 'fea8ed66-7099-4d14-ae45-fb3d468bc7c0', 'cell_id': 'W2sZmlsZQ=='},
  'outputs': ({'output_type': 'stream', 'name': 'stdout', 'text': '1\n'},),
  'execution_count': 2
}
```

In [194]:
nb.by_type('code')

(#38) [1,2,3,4,5,6,7,8,9,10...]

In [195]:
show_doc(NB.codes)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L212){target="_blank" style="float:right; font-size:smaller"}

### NB.codes



*Cell indices of type `code`*

In [196]:
show_doc(NB.mds)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L214){target="_blank" style="float:right; font-size:smaller"}

### NB.mds



*Cell indices of type `markdown`*

In [197]:
nb.mds

(#2) [0,23]

In [198]:
show_doc(NB.idx2cell)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L216){target="_blank" style="float:right; font-size:smaller"}

### NB.idx2cell

>      NB.idx2cell (cell_type:Optional[Literal['code','markdown']]=None)

*Return dict of indices to cells of type `cell_type`*

In [199]:
nb.idx2cell('code')[2]

> code
```json
{
  'idx': 2,
  'cell_type': 'code',
  'source': '# cell 2\nimport time\n# time.sleep(2)\nprint(1)',
  'metadata': {'brd': 'fea8ed66-7099-4d14-ae45-fb3d468bc7c0', 'cell_id': 'W2sZmlsZQ=='},
  'outputs': ({'output_type': 'stream', 'name': 'stdout', 'text': '1\n'},),
  'execution_count': 2
}
```

In [200]:
show_doc(NB.idx2code)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L218){target="_blank" style="float:right; font-size:smaller"}

### NB.idx2code



*Return dict of indices to cells of type `code`*

In [201]:
show_doc(NB.idx2md)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L220){target="_blank" style="float:right; font-size:smaller"}

### NB.idx2md



*Return dict of indices to cells of type `markdown`*

In [202]:
nb.idx2md[0]

> markdown
```json
{
  'idx': 0,
  'cell_type': 'markdown',
  'source': '# cell 0',
  'metadata': {'brd': 'e7ef522b-0719-42ad-ad1e-e4f055330cbf', 'cell_id': 'W0sZmlsZQ=='}
}
```

In [203]:
show_doc(NB.withOutputs)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L223){target="_blank" style="float:right; font-size:smaller"}

### NB.withOutputs



*Return indices of cells with outputs*

In [204]:
show_doc(NB.idx2outputs)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L225){target="_blank" style="float:right; font-size:smaller"}

### NB.idx2outputs



*Return dict of indices to cells with outputs*

In [205]:
# type: ignore

display(cells[codes[2]].outputs[0])

test_eq(did(cells[codes[2]].outputs[0]), cells[codes[2]].outputs[0].metadata['transient']['display_id'])
test_eq(cells[codes[2]].outputs[0].did, cells[codes[2]].outputs[0].metadata['transient']['display_id'])

```json
{ 'data': {'text/plain': "'cell 3'"},
  'metadata': { 'bridge': {'cell': 3},
                'transient': { 'display_id': '9fd35d8abc854b3d601517c52d79350c'}},
  'output_type': 'display_data'}
```

In [206]:
print(nb.idx2outputs.keys())
display(nb.idx2outputs[1])
nb.idx2outputs[2]

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39])


> code
```json
{
  'idx': 1,
  'cell_type': 'code',
  'source': '# cell 1\nfrom itertools import count\n\nimport ipywidgets as W\nimport matplotlib.pyplot as plt\nfrom bridget.display_helpers import displaydh\nfrom IPython.display import HTML, Image, Javascript, JSON, DisplayHandle, clear_output\ncounter = count()',
  'metadata': {'brd': 'ca92cce1-b16c-487b-9869-899330cec0ac', 'cell_id': 'W1sZmlsZQ=='},
  'outputs': (),
  'execution_count': 1
}
```

> code
```json
{
  'idx': 2,
  'cell_type': 'code',
  'source': '# cell 2\nimport time\n# time.sleep(2)\nprint(1)',
  'metadata': {'brd': 'fea8ed66-7099-4d14-ae45-fb3d468bc7c0', 'cell_id': 'W2sZmlsZQ=='},
  'outputs': ({'output_type': 'stream', 'name': 'stdout', 'text': '1\n'},),
  'execution_count': 2
}
```

In [207]:
(cell := nb.idx2outputs[7])

> code
```json
{
  'idx': 7,
  'cell_type': 'code',
  'source': "# cell 7\ndisplaydh(HTML('cell 7.1'), metadata={'bridge': {'cell': 7.1}})\ndisplaydh(HTML('cell 7.2'), metadata={'bridge': {'cell': 7.2}});",
  'metadata': {'brd': 'e43f0a76-d4e3-499f-900a-390b4b877044', 'cell_id': 'X10sZmlsZQ=='},
  'outputs': (
    {
      'output_type': 'display_data',
      'data': {'text/html': 'cell 7.1', 'text/plain': '<IPython.core.display.HTML object>'},
      'metadata': {'transient': {'display_id': 'eb8906f41a8c5b15e9d09ec36cadf9bd'}, 'bridge': {'cell': 7.1}}
    },
    {
      'output_type': 'display_data',
      'data': {'text/html': 'cell 7.2', 'text/plain': '<IPython.core.display.HTML object>'},
      'metadata': {'transient': {'display_id': '0b3856420dda423aa16aa308054a5353'}, 'bridge': {'cell': 7.2}}
    }
  ),
  'execution_count': 7
}
```

In [208]:
def get_data(cell: NBCell, mime:str|None=None): 
    return d.get(mime, None) if (d := nested_idx(cell, 'outputs', 0, 'data')) and mime else d
get_html = partial(get_data, mime='text/html')
get_plain = partial(get_data, mime='text/plain')

print(f"{get_data(cell)=}\n{get_html(cell)=}\n{get_plain(cell)=}")


get_data(cell)={'text/html': 'cell 7.1', 'text/plain': '<IPython.core.display.HTML object>'}
get_html(cell)='cell 7.1'
get_plain(cell)='<IPython.core.display.HTML object>'


In [209]:
show_doc(NB.process)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L238){target="_blank" style="float:right; font-size:smaller"}

### NB.process

>      NB.process
>                  (cbs:Union[olio.callback.Callback,Sequence[olio.callback.Call
>                  back]]=(), slc:slice|None=None,
>                  pred:Optional[Callable[[__main__.NBCell],bool]]=None,
>                  **kwargs)

*Process a subset `slc` of cells filtered by `pred` with `cbs` and `FuncCB` callbacks.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| cbs | Union | () |  |
| slc | slice \| None | None |  |
| pred | Optional | None |  |
| kwargs |  |  |  |
| **Returns** | **NB** |  | **FuncCB kwargs** |

In [210]:
nb.process(on_iter=lambda _,item: print(item.cell_type, end=', '));

markdown, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, markdown, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, 

In [211]:
ctyps = []
cb = FuncCB(on_iter=lambda _,item: ctyps.append(item.cell_type))
process_(nb.cells, cb)
test_eq(ctyps, [c.cell_type for c in nb.cells])

ctyps = []
nb.process(slc=slice(0,2), on_iter=lambda _,item: ctyps.append(item.cell_type))
test_eq(ctyps, ('markdown', 'code'))


In [212]:
nb.process(on_iter=lambda _,item: print((_.n, item.cell_type), end=', '));


(0, 'markdown'), (1, 'code'), (2, 'code'), (3, 'code'), (4, 'code'), (5, 'code'), (6, 'code'), (7, 'code'), (8, 'code'), (9, 'code'), (10, 'code'), (11, 'code'), (12, 'code'), (13, 'code'), (14, 'code'), (15, 'code'), (16, 'code'), (17, 'code'), (18, 'code'), (19, 'code'), (20, 'code'), (21, 'code'), (22, 'code'), (23, 'markdown'), (24, 'code'), (25, 'code'), (26, 'code'), (27, 'code'), (28, 'code'), (29, 'code'), (30, 'code'), (31, 'code'), (32, 'code'), (33, 'code'), (34, 'code'), (35, 'code'), (36, 'code'), (37, 'code'), (38, 'code'), (39, 'code'), 

In [213]:
def _enumerate(istat, item:NBCell): 
    item.metadata['n'] = istat.n

test_eq(vals_at(nb.cells, '*.metadata.n'), empty)
nb.process(on_iter=_enumerate);
test_eq(vals_at(nb.cells, '*.metadata.n'), range(len(nb.cells)))


In [214]:
show_doc(NB.pipe)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L250){target="_blank" style="float:right; font-size:smaller"}

### NB.pipe

>      NB.pipe (funcs:Iterable[__main__.NBProcessor], *args, **kwargs)

*Pipe a sequence of `NBProcessor`s*

In [215]:
nb.pipe((lambda nb: NB.from_NB(nb, nb.cells[nb.mds]),)).cells.attrgot('source')


(#2) ['# cell 0','# cell 23']

# Diffs

In [216]:
#| export

@FC.patch
def check(self:NB):
    assert self.nbData['cellCount'] == len(self.cells)
    for n, c in enumerate(self.cells):
        assert c.idx == n

In [217]:
state = json.loads(Path('../packages/nbinspect-vscode/test/outputs.json').read_text('utf-8'))

nb = NB.fromStateMessage(state)
len(nb.cells)

40

In [218]:
diffs1 = json.loads(Path('../packages/nbinspect-vscode/test/update01.json').read_text('utf-8'))

RenderJSON(diffs1, init_level=4, max_height=400).display()

In [219]:
def apply_diff(nb:NB, diff):
    changed, added, removed, cellCount = gets(diff, 'cells', 'added', 'removed', 'cellCount')
    cells = nb.cells
    for cell in changed:  # type: ignore
        cell = NBCell(cell)
        cells[cell.idx] = cell

apply_diff(nb, diffs1['changes'][0])
nb.check()

In [220]:
diffs2 = json.loads(Path('../packages/nbinspect-vscode/test/update13.json').read_text('utf-8'))

RenderJSON(diffs2, init_level=5, max_height=400).display()

In [221]:
def apply_diff(nb:NB, diff):
    changed, added, removed, cellCount = gets(diff, 'cells', 'added', 'removed', 'cellCount')
    cells = nb.cells
    if removed:
        for idx in sorted(removed, reverse=True):  # type: ignore
            del cells[idx]
            for cell in cells[idx:]:
                cell.idx -= 1
    nb.nbData['cellCount'] = cellCount
    for cell in changed:  # type: ignore
        cells[cell.idx] = NBCell(cell)

nb = NB.fromStateMessage(state)
apply_diff(nb, diffs2['changes'][0])
nb.check()

In [222]:
#| export

@FC.patch
def _apply_diff(self:NB, diff):
    cells = self.cells
    changed, added, removed, cellCount = gets(diff, 'cells', 'added', 'removed', 'cellCount')
    if isinstance(removed, list) and removed:
        for idx in sorted(removed, reverse=True):  # type: ignore
            del cells[idx]
            for cell in cells[idx:]: cell.idx -= 1
    elif isinstance(added, list) and added:
        for cell in added:  # type: ignore
            idx = cell['idx']
            cells.insert(idx, NBCell(cell))
            for cell in cells[idx+1:]: cell.idx += 1
    for cell in changed: cells[cell['idx']] = NBCell(cell)  # type: ignore
    self.nbData['cellCount'] = cellCount

In [223]:
diffs3 = json.loads(Path('../packages/nbinspect-vscode/test/update14.json').read_text('utf-8'))

RenderJSON(diffs3, init_level=5, max_height=400).display()

In [224]:
nb._apply_diff(diffs3['changes'][0])
nb.check()

In [225]:
#| export

@FC.patch
def apply_diffsMessage(self:NB, diffs):
    for diff in diffs['changes']: self._apply_diff(diff)
    self.check()

In [226]:
nb = NB.fromStateMessage(state)

ff = sorted(Path('../packages/nbinspect-vscode/test/').glob('update*.json'))
for f in ff[:12]:
    diffs = json.loads(f.read_text('utf-8'))
    nb.apply_diffsMessage(diffs)

In [227]:
nb = NB.fromStateMessage(state)

for f in ff[12:]:
    diffs = json.loads(f.read_text('utf-8'))
    nb.apply_diffsMessage(diffs)

# IpynbConvertCB
> Notebook state to IPython notebook format (ipynb)

NBCell and friends are mostly [Notebook format types](https://nbformat.readthedocs.io/en/4.4.0/format_description.html#cell-types) without enforcing some attributes. Going back and forth between the two formats is easy.

In [228]:
#| export

class IpynbOutput(AD): ...
class IpynbCell(AD): outputs: tuple[IpynbOutput, ...]

class St2Ipynb(FC.Transform): 
    def encodes(self, x: NBOutput) -> IpynbOutput:  # type: ignore
        fmt = IpynbOutput(x)
        if fmt['output_type'] in ('display_data', 'execute_result'): fmt.metadata=x.get('metadata', {})
        fmt.get('metadata', emptyd).pop('transient', None)
        return fmt
    def decodes(self, x: IpynbOutput) -> NBOutput:  # type: ignore
        return NBOutput(x)

    def encodes(self, x: NBCell) -> IpynbCell:
        fmt = update_(IpynbCell(x), metadata=x.get('metadata', {}))
        fmt.pop('idx', None)
        if 'outputs' in x: fmt['outputs'] = self(x.outputs)
        # if x['cell_type'] == 'code': 
        #     fmt['execution_count'] = fmt.get('execution_count', None)
        return fmt
    def decodes(self, x: IpynbCell) -> NBCell:
        return NBCell(x)


In [229]:
cell = NBCell({
    'cell_type': 'code', 'source': '',
    'outputs': ({
        'output_type': 'display_data',
        'data': {'text/plain': ''},
        'metadata': {'transient': {'display_id': 'abc'}}
    },)
})
test_eq(type(c := cast(IpynbCell, St2Ipynb()(cell))), IpynbCell)
test_eq(c.outputs[0].metadata, {})


In [230]:
icells = nb.cells.map(St2Ipynb())
RenderJSON(icells, init_level=1, max_height=400).display()


In [231]:
#| export

class IpynbConvertCB(Callback):
    def before_iter(self, istat): 
        nb = istat.context
        d, ks, mdks = nb.nbData['metadata'], ('nbformat', 'nbformat_minor'), ('kernelspec', 'language_info')
        self.notebook = {
            'cells': [],
            **FC.filter_keys(d, FC.in_(ks)),  # type: ignore
            'metadata': FC.filter_keys(d['metadata'], FC.in_(mdks)),  # type: ignore
        }
        self.cell_counter = 0
        self._fmt = St2Ipynb()
    
    def on_iter(self, _, cell):
        # fmt = update_(St2Ipynb()(cell), id=self.cell_counter)
        self.notebook['cells'].append(self._fmt(cell))
        self.cell_counter += 1


In [232]:
# processor = BridgetNBProcessor(state)
nb.process(cb := IpynbConvertCB())
ipynb_json = cb.notebook

nb = nbformat.reads(json.dumps(ipynb_json), as_version=nbformat.NO_CONVERT, capture_validation_error=(derr := {}))
test_eq(derr, {})

RenderJSON(ipynb_json, init_level=1, max_height=400).display()


# MDConvertCB


# StateConversionCB

In [233]:
# class StateConversionCB(Callback):
#     @cached_property
#     def state(self) -> AD: return AD(type='state',cells=[])
    
#     def _convert_output(self, output):
#         converted = dict(output)
#         if 'output_type' in converted:
#             converted['metadata']['outputType'] = converted.pop('output_type')
#         return converted
    
#     def on_iter(self, ctx, cell):
#         # Convert each cell back to the original state format
#         st_cell = NBCell({
#             'cell_type': cell.cell_type,
#             'source': ''.join(cell.source) if isinstance(cell.source, list) else cell.source,
#         })
#         if cell.metadata: st_cell.metadata = cell.metadata
#         if cell.cell_type == 'code':
#             st_cell['outputs'] = [self._convert_output(o) for o in cell.outputs]
#         self.state.cells.append(st_cell)

# tracker = CollBack(nb.cells, cbs=[cb := StateConversionCB()])
# tracker.update(item=nb.cells[0])
# test_eq(cell, nb.cells[0])

# # with tracker.this_cbs([cb := StateConversionCB()]):
# #     # for cell in tracker: pass
# #     cell = next(iter(tracker))
# # converted_state = cb.state

# # # Compare with original state
# # for i, c1,c2 in zip(range(len(converted_state.cells)), converted_state.cells, AD(state).cells):
# #     cprint(c1, c2)
# #     test_eq(c1, c2)
# # # assert converted_state['cells'] == state['cells'], "Round-trip conversion failed"

# Find

In [234]:
class FindCB(Callback):
    def __init__(self, what:str|Callable[[NBCell], bool], where:str='source'): 
        self.what = (lambda c: what in getattr(c, where)) if isinstance(what, str) else what
        self._cells = []
    @property
    def nb(self): return NB(self._cells)
    def on_iter(self, _, cell):
        if has_directive(cell, 'hide'): return
        if self.what(cell): self._cells.append(cell)


In [235]:
nb = NB.fromStateMessage(state)


In [236]:
cb = FindCB('displaydh')
shortens(nb.process(cb).cells, 'r', 80)

["{'idx': 1, 'cell_type': 'code', 'source': '# cell 1\\nfrom itertools import count…",
 '{\'idx\': 3, \'cell_type\': \'code\', \'source\': "# cell 3\\ndisplaydh(\'cell 3\', metadat…',
 '{\'idx\': 4, \'cell_type\': \'code\', \'source\': "# cell 4\\ndisplaydh(HTML(\'cell 4\'), m…',
 "{'idx': 5, 'cell_type': 'code', 'source': '# cell 5\\ndisplaydh(Javascript(\\'cons…",
 '{\'idx\': 7, \'cell_type\': \'code\', \'source\': "# cell 7\\ndisplaydh(HTML(\'cell 7.1\'),…',
 '{\'idx\': 16, \'cell_type\': \'code\', \'source\': "# cell 16\\ndh = displaydh(display_id…',
 '{\'idx\': 17, \'cell_type\': \'code\', \'source\': "# cell 17\\ndh = displaydh(HTML(\'cell…',
 '{\'idx\': 19, \'cell_type\': \'code\', \'source\': "# cell 19\\ndisplaydh(JSON({\'cell\': 1…',
 '{\'idx\': 24, \'cell_type\': \'code\', \'source\': "# cell 24\\nfrom itertools import cou…',
 '{\'idx\': 26, \'cell_type\': \'code\', \'source\': "# cell 26\\n_ = displaydh(\'cell 26.1\'…',
 '{\'idx\': 33, \'cell_type\': \'code\', \'source\': "

In [237]:
list(nb.cells.filter(lambda c: 'displaydh' in c.source).map(shorten, mode='r', limit=80))


["{'idx': 1, 'cell_type': 'code', 'source': '# cell 1\\nfrom itertools import count…",
 '{\'idx\': 3, \'cell_type\': \'code\', \'source\': "# cell 3\\ndisplaydh(\'cell 3\', metadat…',
 '{\'idx\': 4, \'cell_type\': \'code\', \'source\': "# cell 4\\ndisplaydh(HTML(\'cell 4\'), m…',
 "{'idx': 5, 'cell_type': 'code', 'source': '# cell 5\\ndisplaydh(Javascript(\\'cons…",
 '{\'idx\': 7, \'cell_type\': \'code\', \'source\': "# cell 7\\ndisplaydh(HTML(\'cell 7.1\'),…',
 '{\'idx\': 16, \'cell_type\': \'code\', \'source\': "# cell 16\\ndh = displaydh(display_id…',
 '{\'idx\': 17, \'cell_type\': \'code\', \'source\': "# cell 17\\ndh = displaydh(HTML(\'cell…',
 '{\'idx\': 19, \'cell_type\': \'code\', \'source\': "# cell 19\\ndisplaydh(JSON({\'cell\': 1…',
 '{\'idx\': 24, \'cell_type\': \'code\', \'source\': "# cell 24\\nfrom itertools import cou…',
 '{\'idx\': 26, \'cell_type\': \'code\', \'source\': "# cell 26\\n_ = displaydh(\'cell 26.1\'…',
 '{\'idx\': 33, \'cell_type\': \'code\', \'source\': "

In [238]:
list(nb.source.filter(lambda c: 'displaydh' in c))


['# cell 1\nfrom itertools import count\n\nimport ipywidgets as W\nimport matplotlib.pyplot as plt\nfrom bridget.display_helpers import displaydh\nfrom IPython.display import HTML, Image, Javascript, JSON, DisplayHandle, clear_output\ncounter = count()',
 "# cell 3\ndisplaydh('cell 3', metadata={'bridge': {'cell': 3}});",
 "# cell 4\ndisplaydh(HTML('cell 4'), metadata={'bridge': {'cell': 4}});",
 '# cell 5\ndisplaydh(Javascript(\'console.log("cell 5")\'), metadata={\'bridge\': {\'cell\': 5}});',
 "# cell 7\ndisplaydh(HTML('cell 7.1'), metadata={'bridge': {'cell': 7.1}})\ndisplaydh(HTML('cell 7.2'), metadata={'bridge': {'cell': 7.2}});",
 "# cell 16\ndh = displaydh(display_id=True, metadata={'bridge': {'cell': 16}})",
 "# cell 17\ndh = displaydh(HTML('cell 17'), metadata={'bridge': {'cell': 17}})",
 "# cell 19\ndisplaydh(JSON({'cell': 19}), metadata={'bridge': {'cell': 19}});",
 "# cell 24\nfrom itertools import count\nfrom bridget.display_helpers import displaydh\nfrom IPython.display 

In [239]:
cb = FindCB('# find_me')
c = nb.process(cb).cells[0]
c

> code
```json
{
  'idx': 39,
  'cell_type': 'code',
  'source': '# cell 39\n# find_me',
  'metadata': {'brd': '064cdd49-667e-4da9-b001-4755659aeb38', 'cell_id': 'X54sZmlsZQ=='},
  'outputs': (),
  'execution_count': None
}
```

In [240]:
WhereT = Literal['source', 'outputs', 'metadata', 'all']

@FC.patch
def find(self: NB, what:str|Callable[[NBCell], bool], where:WhereT='source') -> L:
    if isinstance(what, str): f = (lambda c: what in (str(getattr(c, where, '') if where != 'all' else c)))
    else: f = what
    return self.cells.filter(f)


In [241]:
nb.find('# find_me')


(#1) [{'idx': 39, 'cell_type': 'code', 'source': '# cell 39\n# find_me', 'metadata': {'brd': '064cdd49-667e-4da9-b001-4755659aeb38', 'cell_id': 'X54sZmlsZQ=='}, 'outputs': (), 'execution_count': None}]

In [242]:
val_at(c, 'metadata.brd')


'064cdd49-667e-4da9-b001-4755659aeb38'

In [243]:
c = nb.cells[4]
c

> code
```json
{
  'idx': 4,
  'cell_type': 'code',
  'source': "# cell 4\ndisplaydh(HTML('cell 4'), metadata={'bridge': {'cell': 4}});",
  'metadata': {'brd': '425ba617-db53-4918-958e-1a355849a808', 'cell_id': 'W4sZmlsZQ=='},
  'outputs': (
    {
      'output_type': 'display_data',
      'data': {'text/html': 'cell 4', 'text/plain': '<IPython.core.display.HTML object>'},
      'metadata': {'bridge': {'cell': 4}}
    },
  ),
  'execution_count': 4
}
```

In [244]:
vals_at(c, 'outputs.*.data.text/plain')


('<IPython.core.display.HTML object>',)

In [245]:
nb.cells[2]

> code
```json
{
  'idx': 2,
  'cell_type': 'code',
  'source': '# cell 2\nimport time\n# time.sleep(2)\nprint(1)',
  'metadata': {'brd': 'fea8ed66-7099-4d14-ae45-fb3d468bc7c0', 'cell_id': 'W2sZmlsZQ=='},
  'outputs': ({'output_type': 'stream', 'name': 'stdout', 'text': '1\n'},),
  'execution_count': 2
}
```

In [246]:
vals_at(nb.cells, '*.outputs.*.data.text/plain')


(empty,
 empty,
 empty,
 ("'cell 3'",),
 ('<IPython.core.display.HTML object>',),
 ('<IPython.core.display.Javascript object>',),
 empty,
 ('<IPython.core.display.HTML object>', '<IPython.core.display.HTML object>'),
 ('<IPython.core.display.HTML object>',),
 ('<IPython.core.display.Javascript object>',),
 ('<IPython.core.display.Markdown object>',),
 ('<IPython.core.display.SVG object>',),
 ("IntSlider(value=12, description='cell')",),
 ('Output()',),
 ('<Figure size 400x267 with 1 Axes>',),
 ('<IPython.core.display.Image object>',),
 empty,
 ('<IPython.core.display.HTML object>',),
 empty,
 ('<IPython.core.display.JSON object>',),
 ("IntSlider(value=20, description='cell')",),
 empty,
 empty,
 empty,
 ('<IPython.core.display.HTML object>', ''),
 empty,
 ("'cell 26.1'", "'cell 26.2'"),
 ("'cell 27.1'", "'cell 27.2'"),
 empty,
 empty,
 empty,
 empty,
 empty,
 empty,
 empty,
 ('Output()',),
 empty,
 empty,
 empty,
 empty)

In [247]:
vals_at(nb.cells, '*.source')


('# cell 0',
 '# cell 1\nfrom itertools import count\n\nimport ipywidgets as W\nimport matplotlib.pyplot as plt\nfrom bridget.display_helpers import displaydh\nfrom IPython.display import HTML, Image, Javascript, JSON, DisplayHandle, clear_output\ncounter = count()',
 '# cell 2\nimport time\n# time.sleep(2)\nprint(1)',
 "# cell 3\ndisplaydh('cell 3', metadata={'bridge': {'cell': 3}});",
 "# cell 4\ndisplaydh(HTML('cell 4'), metadata={'bridge': {'cell': 4}});",
 '# cell 5\ndisplaydh(Javascript(\'console.log("cell 5")\'), metadata={\'bridge\': {\'cell\': 5}});',
 '# cell 6\nprint(6.1)\nprint(6.2)',
 "# cell 7\ndisplaydh(HTML('cell 7.1'), metadata={'bridge': {'cell': 7.1}})\ndisplaydh(HTML('cell 7.2'), metadata={'bridge': {'cell': 7.2}});",
 '%HTML\n<h3>cell 8</h3>',
 "%javascript\nconsole.log('cell 9')",
 '%markdown\ncell 10',
 '%SVG\n<svg width="100" height="20" xmlns="http://www.w3.org/2000/svg">\n  <text x="10" y="10">cell 11</text>\n</svg>',
 "# cell 12\nW.IntSlider(12, description='

In [248]:
vals_at(nb.cells, '*.metadata')

({'brd': 'e7ef522b-0719-42ad-ad1e-e4f055330cbf', 'cell_id': 'W0sZmlsZQ=='},
 {'brd': 'ca92cce1-b16c-487b-9869-899330cec0ac', 'cell_id': 'W1sZmlsZQ=='},
 {'brd': 'fea8ed66-7099-4d14-ae45-fb3d468bc7c0', 'cell_id': 'W2sZmlsZQ=='},
 {'brd': 'asdf-qwer', 'cell_id': 'W3sZmlsZQ=='},
 {'brd': '425ba617-db53-4918-958e-1a355849a808', 'cell_id': 'W4sZmlsZQ=='},
 {'brd': '906bad9f-0913-48c5-9b70-6ae49a9379c8', 'cell_id': 'W5sZmlsZQ=='},
 {'brd': 'ad631ab3-f31e-412e-8f92-ecca8154a728', 'cell_id': 'W6sZmlsZQ=='},
 {'brd': 'e43f0a76-d4e3-499f-900a-390b4b877044', 'cell_id': 'X10sZmlsZQ=='},
 {'brd': 'ef5efcef-ca2e-4297-9630-37e470020d32', 'cell_id': 'X11sZmlsZQ=='},
 {'brd': '586aa663-4229-4780-9d04-d578ac99e549', 'cell_id': 'X12sZmlsZQ=='},
 {'brd': '3544f667-4bea-4c77-b272-86fc9913a93d', 'cell_id': 'X13sZmlsZQ=='},
 {'brd': '1fad9c73-4425-4dcb-937f-e01084458772', 'cell_id': 'X14sZmlsZQ=='},
 {'brd': '8b29b413-e054-4a61-8ab9-db8031064cde', 'cell_id': 'X15sZmlsZQ=='},
 {'brd': '88073935-1666-4afc-832c

In [249]:
vals_at(nb.cells, '*.outputs')[10:15]  # type: ignore

(({'output_type': 'display_data',
   'data': {'text/markdown': 'cell 10\n',
    'text/plain': '<IPython.core.display.Markdown object>'},
   'metadata': {}},),
 ({'output_type': 'display_data',
   'data': {'image/svg+xml': '<svg xmlns="http://www.w3.org/2000/svg" width="100" height="20">\n  <text x="10" y="10">cell 11</text>\n</svg>',
    'text/plain': '<IPython.core.display.SVG object>'},
   'metadata': {'__displayOpenPlotIcon': True}},),
 ({'output_type': 'execute_result',
   'data': {'application/vnd.jupyter.widget-view+json': {'version_major': 2,
     'version_minor': 0,
     'model_id': 'e167426b9a62409f9e67edc703a25e53'},
    'text/plain': "IntSlider(value=12, description='cell')"},
   'execution_count': 12,
   'metadata': {}},),
 ({'output_type': 'display_data',
   'data': {'application/vnd.jupyter.widget-view+json': {'version_major': 2,
     'version_minor': 0,
     'model_id': '7fb860b9646849fc9a2c7a77f71abf74'},
    'text/plain': 'Output()'},
   'metadata': {}},),
 ({'output_t

## find


In [250]:
#| exporti

def _there(where, what, op, c):
    res = vals_at(c, where)
    if res is empty: return False
    else: return op(res, what) if '*' not in where else any(op(r, what) for r in res)  # type: ignore


In [251]:
_there('outputs.*.data', 'stdout', op.contains, nb.cells[7])

False

In [252]:
_there('outputs.0.name', 'stdout', op.contains, nb.cells[2])

True

In [253]:
_there('metadata.bridget.skip', True, op.is_, nb.cells[24])

False

In [254]:
_there('source', '# find_me', op.contains, nb.cells[32])

False

In [255]:
#| export

JSONVal: TypeAlias = str | int | float | bool | None | dict[str, 'JSONVal'] | list['JSONVal']

@FC.patch
def find(self: NB, what:JSONVal|Callable[[NBCell], bool], where:str='source', op=op.contains) -> L:
    "Find cells matching `what` in `where` using `op`"
    # if isinstance(what, str): what = lambda c: False if (res := _F(c, where, None)) is None else what in res
    if not isinstance(what, Callable): what = partial(_there, where, what, op)
    return self.cells.filter(what)


In [256]:
nbp = nb.process(slc=slice(31, None))
# nbp.find('# find_me').attrgot('source').map(shorten, mode='r')
vals_at(nbp.find('# find_me'), '*.source')


('# cell 39\n# find_me',)

In [257]:
nb.find(True, 'metadata.bridget.skip', op.is_).attrgot('metadata')


(#0) []

In [258]:
nb.find('stdout', 'outputs.0.name').attrgot('source')


(#2) ['# cell 2\nimport time\n# time.sleep(2)\nprint(1)','# cell 6\nprint(6.1)\nprint(6.2)']

In [259]:
nb.find('stdout', 'outputs.*.name').attrgot('source')


(#2) ['# cell 2\nimport time\n# time.sleep(2)\nprint(1)','# cell 6\nprint(6.1)\nprint(6.2)']

## found

In [260]:
@FC.patch
def found(self: NB, what:str|Callable[[NBCell], bool], where:WhereT='source', cbs:Callback|Sequence[Callback]=()) -> NB:
    return self.process(cbs or FindCB(what, where))

nb.found('# find_me').cells


(#1) [{'idx': 39, 'cell_type': 'code', 'source': '# cell 39\n# find_me', 'metadata': {'brd': '064cdd49-667e-4da9-b001-4755659aeb38', 'cell_id': 'X54sZmlsZQ=='}, 'outputs': (), 'execution_count': None}]

In [261]:
#| export

@FC.patch
def found(self: NB, 
        what:JSONVal|Callable[[NBCell], bool], where='source', op=op.contains, 
        cbs:Callback|Sequence[Callback]=()
    ) -> NB:
    return NB.from_NB(self, self.find(what, where, op)).process(cbs)


In [262]:
nb.found('# find_me').cells.attrgot('source')


(#1) ['# cell 39\n# find_me']

In [263]:
nb.found('#| hide').cells

(#1) [{'idx': 13, 'cell_type': 'code', 'source': "#| hide\n# cell 13\nw = W.Output()\ndisplay(w)\nwith w:\n    print('cell 13.1')\n    display('cell 13.2')\n    display(HTML('cell 13.3'))", 'metadata': {'brd': '88073935-1666-4afc-832c-c952fd7638bf', 'cell_id': 'X16sZmlsZQ=='}, 'outputs': ({'output_type': 'display_data', 'data': {'application/vnd.jupyter.widget-view+json': {'version_major': 2, 'version_minor': 0, 'model_id': '7fb860b9646849fc9a2c7a77f71abf74'}, 'text/plain': 'Output()'}, 'metadata': {}},), 'execution_count': 13}]

In [264]:
class StripDirectives(Callback):
    @property
    def nb(self): return self._nb
    def before_iter(self, istat): self._nb = NB.from_NB(istat.context, [])
    def on_iter(self, _, cell):
        self._nb.cells.append(c := cell.copy())
        extract_directives(c)

display(shortens(nb.found('#| hide').source, 'r'))
shortens(nb.found('#| hide', cbs=StripDirectives()).source, 'r')


['#| hide\n# cell 13\nw = W.Output()\ndisplay…']

['# cell 13\nw = W.Output()\ndisplay(w)\nwith…']

In [265]:
def strip_nbdev_dirs_(nb:NB):
    f = FC.compose(copycell, TZ.curried.do(extract_directives))
    return NB.from_NB(nb, nb.cells.map(f))

shortens(nb.found('#| hide').pipe((strip_nbdev_dirs_,)).source, 'r')


['# cell 13\nw = W.Output()\ndisplay(w)\nwith…']

# Colophon
----


In [266]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean


In [267]:
if FC.IN_NOTEBOOK:
    nb_path = '20_nb.ipynb'
    # nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)
